# Dask-Snowflake Integration Demo

This notebook demonstrates the `dask-snowflake` integration package which supports parallel read/write from Snowflake to Python with Dask.

The notebook is structured as follows:
1. Setup remote Dask resources
2. Connect to Snowflake
3. Write Data to Snowflake in Parallel
4. Read Data from Snowflake in Parallel
5. Use Dask to Train XGBoost on Snowflake Data

In [34]:
import os
import time

## 1. Set-up Dask Resources

We'll start by launching our local Dask.

In [35]:
from dask.distributed import Client, wait, get_worker, LocalCluster

In [36]:
cluster = LocalCluster(threads_per_worker=8, n_workers=2)
client = Client(cluster)
client.scheduler_info()

/opt/conda/envs/rapids/lib/python3.8/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41553 instead
  warnings.warn(


{'type': 'Scheduler',
 'id': 'Scheduler-0cfb5b8a-8ed3-4377-953e-c34f1f2ebdf1',
 'address': 'tcp://127.0.0.1:43631',
 'services': {'dashboard': 41553},
 'started': 1651166191.9158804,
 'workers': {'tcp://127.0.0.1:33297': {'type': 'Worker',
   'id': 1,
   'host': '127.0.0.1',
   'resources': {},
   'local_directory': '/rapids/notebooks/snowflake/dask-worker-space/worker-um3ubexp',
   'name': 1,
   'nthreads': 8,
   'memory_limit': 270328014848,
   'last_seen': 1651166192.4743316,
   'services': {'dashboard': 40875},
   'metrics': {'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}},
    'spilled_nbytes': {'memory': 0, 'disk': 0},
    'cpu': 0.0,
    'memory': 68259840,
    'time': 1651166192.2971048,
    'read_bytes': 0.0,
    'write_bytes': 0.0,
    'read_bytes_disk': 0.0,
    'write_bytes_disk': 0.0,
    'num_fds': 26,
    'gpu_utilization': 0,
    'gpu_memory_used': 8223129600,
    'gpu': {'utilization': 0, 'memory-used': 8223129600}},
   'nanny': 'tcp://127.0.0.1:41395',
   'gpu': {'memory-total': 51527024640, 'name': 'NVIDIA RTX A6000'}},
  'tcp://127.0.0.1:38405': {'type': 'Worker',
   'id': 0,
   'host': '127.0.0.1',
   'resources': {},
   'local_directory': '/rapids/notebooks/snowflake/dask-worker-space/worker-1mw2_fy2',
   'name': 0,
   'nthreads': 8,
   'memory_limit': 270328014848,
   'last_seen': 1651166192.4936323,
   'services': {'dashboard': 44759},
   'metrics': {'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}},
    'spilled_nbytes': {'memory': 0, 'disk': 0},
    'cpu': 0.0,
    'memory': 68403200,
    'time': 1651166192.3115957,
    'read_bytes': 0.0,
    'write_bytes': 0.0,
    'read_bytes_disk': 0.0,
    'write_bytes_disk': 0.0,
    'num_fds': 26,
    'gpu_utilization': 0,
    'gpu_memory_used': 8223129600,
    'gpu': {'utilization': 0, 'memory-used': 8223129600}},
   'nanny': 'tcp://127.0.0.1:39191',
   'gpu': {'memory-total': 51527024640, 'name': 'NVIDIA RTX A6000'}}}}

## 2. Connect to Snowflake
Let's now connect our Python session to Snowflake using Snowflake's connector. 

**NOTE:** For this section to work you will need to have the Snowflake Sample Data available in your account. Otherwise, change the query to something relevant/applicable to your use case.

In [37]:
from dotenv import load_dotenv

config = load_dotenv()

DB_CREDS = {
    "user": os.environ["SNOW_USER"],
    "password": os.environ["SNOW_PASSWORD"],
    "account": os.environ["SNOW_ACCOUNT"],
    "database": "SAMPLE_DATA",
    "warehouse": os.environ["SNOW_WAREHOUSE"]
}

In [38]:
import snowflake.connector

In [39]:
# create Snowflake Python connector
ctx = snowflake.connector.connect(
    user=os.environ["SNOW_USER"],
    password=os.environ["SNOW_PASSWORD"],
    account=os.environ["SNOW_ACCOUNT"],
    warehouse=os.environ["SNOW_WAREHOUSE"]
)

And run some sample code to test the connection:

In [40]:
# run sample code to test connection
cs = ctx.cursor()

schema = "TPCDS_SF100TCL"
table = "CALL_CENTER"

cs.execute("USE SAMPLE_DATA")
cs.execute("SELECT * FROM " + schema + "." + table)

one_row = str(cs.fetchone())

print(one_row)

(1, 'AAAAAAAABAAAAAAA', datetime.date(1998, 1, 1), None, None, 2450952, 'NY Metro', 'large', 597159671, 481436415, '8AM-4PM', 'Bob Belcher', 6, 'More than other authori', 'Shared others could not count fully dollars. New members ca', 'Julius Tran', 3, 'pri', 6, 'cally', '730', 'Ash Hill', 'Boulevard', 'Suite 0', 'Georgetown', 'Harmon County', 'OK', '77057', 'United States', Decimal('-6.00'), Decimal('0.11'))


## 3. Parallel Write to Snowflake

Now that we have launched our remote compute resources and tested our connection to Snowflake, let's generate some synthetic data with Dask and then write to a Snowflake database in parallel.

In [41]:
import dask

In [42]:
# generate synthetic timeseries data
ddf = dask.datasets.timeseries(
    start="2021-01-01",
    end="2021-01-10",
)
len(ddf)

777600

In [43]:
# create warehouse and database
cs.execute("CREATE WAREHOUSE IF NOT EXISTS dask_snowflake_wh")
cs.execute("CREATE DATABASE IF NOT EXISTS dask_snowflake_db")
cs.execute("USE DATABASE dask_snowflake_db")

In [45]:
from dask_snowflake import to_snowflake

In [46]:
connection_kwargs = {
    "user": os.environ["SNOW_USER"],
    "password": os.environ["SNOW_PASSWORD"],
    "account": os.environ["SNOW_ACCOUNT"],
    "warehouse": os.environ["SNOW_WAREHOUSE"],
    "database": "dask_snowflake_db",
    "schema": "PUBLIC",
}

In [47]:
%%time
# write Dask dataframe to Snowflake in parallel
to_snowflake(
    ddf,
    name="dask_snowflake_table",
    connection_kwargs=connection_kwargs,
)

CPU times: user 11.3 s, sys: 1.35 s, total: 12.6 s
Wall time: 1min 14s


## 4. Parallel Read from Snowflake
We can now read this data back into our Python session in parallel.

In [48]:
from dask_snowflake import read_snowflake

In [49]:
%%time
# read data from snowflake into a Dask dataframe
snowflake_data = read_snowflake(
    query="""
      SELECT *
      FROM dask_snowflake_table;
   """,
    connection_kwargs=connection_kwargs,
)

print(snowflake_data.head())

     ID     NAME         X         Y
0   954   Yvonne -0.375479 -0.062307
1   985    Edith  0.171367  0.957500
2   975  Norbert -0.064329 -0.082382
3  1000      Ray  0.449260 -0.091000
4   989      Ray  0.139058  0.063518
CPU times: user 384 ms, sys: 47.3 ms, total: 431 ms
Wall time: 2.17 s


In [50]:
len(snowflake_data)

777600

In [51]:
# perform computation over Snowflake data with Dask
result = snowflake_data.X.mean()
print(result.compute())

-0.0018834459030034863


## 5. Machine Learning
After loading data into our Python session from Snowflake, we can use Python for what it's good at: things like free-form, iterative exploratory analyses and complex Machine Learning models.

Let's read in some data from Snowflake using the `dask-snowflake` connector and then train an XGBoost ML model on that data.

In [ ]:
# define schema and query
SCHEMA = "SAMPLE_DATA.TPCH_SF100"

example_query=f"""
SELECT
    
    C_CUSTKEY,
    C_NAME,
    SUM(L_QUANTITY) AS sum_qty,
    SUM(PS_AVAILQTY) AS sum_avail_qty,
    MAX(P_RETAILPRICE) AS max_retail_price
    
    FROM {SCHEMA}.CUSTOMER
    
        JOIN {SCHEMA}.ORDERS
            ON C_CUSTKEY = O_CUSTKEY
            
            JOIN {SCHEMA}.LINEITEM
                ON L_ORDERKEY = O_ORDERKEY
                
                JOIN {SCHEMA}.PART
                    ON P_PARTKEY = L_PARTKEY
                    
                    JOIN {SCHEMA}.PARTSUPP
                        ON P_PARTKEY = PS_PARTKEY
    
    WHERE PS_SUPPLYCOST > 10

GROUP BY C_CUSTKEY, C_NAME
"""

In [ ]:
# set connection parameters
connection_kwargs = {
    "user": os.environ["SNOW_USER"],
    "password": os.environ["SNOW_PASSWORD"],
    "account": os.environ["SNOW_ACCOUNT"],
    "warehouse": os.environ["SNOW_WAREHOUSE"],
    "database": "SAMPLE_DATA",
    "schema": "TPCH_SF100",
}

In [ ]:
%%time
# read in data from snowflake
ddf = read_snowflake(
    query=example_query,
    connection_kwargs=connection_kwargs,
)

In [ ]:
import xgboost as xgb

In [ ]:
# define predictor and target features
X = ddf[['SUM_AVAIL_QTY', 'MAX_RETAIL_PRICE']]
y = ddf.SUM_QTY

In [ ]:
# create Dask DMatrix
dtrain = xgb.dask.DaskDMatrix(client, X, y)

In [ ]:
%%time
# train XGBoost with Dask
output = xgb.dask.train(
    client,
    {
        'verbosity': 2,
        'tree_method': 'hist',
        'objective': 'reg:squarederror'
    },
    dtrain,
    num_boost_round=10,
    evals=[(dtrain, 'train')]
)

In [ ]:
# make predictions
y_pred = xgb.dask.predict(client, output["booster"], X)
y_pred.compute()

For more details on how to use distributed XGBoost training with Dask, see [this blog post](https://coiled.io/blog/dask-xgboost-python-example/).

# Summary

This notebook has demonstrated:
1. How to use the `dask-snowflake` connector for fast, parallel data transfer between Snowflake and Python
2. How to use Dask to continue manipulating the Snowflake data in a Python session, performing iterative EDA and/or machine learning tasks.

Join the [Dask Discourse](https://dask.discourse.group/) to continue the conversation!